# Amazon AWS S3 Search with LlamaIndex

This example shows how to use the Python [LlamaIndex](https://www.llamaindex.ai/) to run a text-generation request on open-source LLMs and embedding models using the OpenAI SDK, then augment that request using documents stored in an Amazon AWS S3 bucket.

### <u>Requirements</u>
1. As you will accessing the LLMs and embedding models through Vector AI Engineering's Kaleidoscope Service (Vector Inference + Autoscaling), you will need to request a KScope API Key:
- If using **VPN**:
  
  Visit [https://kscope.vectorinstitute.ai/](https://kscope.vectorinstitute.ai/) and select *Request API Key*.
  
- If running **without VPN**:

  Run the following command (replace ```<user_id>``` and ```<password>```) from **within the cluster** to obtain the API Key. The ```access_token``` in the output is your KScope API Key.
  ```bash
  curl -X POST -d "grant_type=password" -d "username=<user_id>" -d "password=<password>" https://kscope.vectorinstitute.ai/token
  ```
2. After obtaining the `.env` configurations, make sure to create the ```.kscope.env``` file in your home directory (```/h/<user_id>```) and set the following env variables:
- For local models through Kaleidoscope (KScope):
    ```bash
    export OPENAI_BASE_URL="https://kscope.vectorinstitute.ai/v1"
    export OPENAI_API_KEY=<kscope_api_key>
    ```
- For OpenAI models:
   ```bash
   export OPENAI_BASE_URL="https://api.openai.com/v1"
   export OPENAI_API_KEY=<openai_api_key>
3. You will also need an Amazon AWS account, with some documents stored in a S3 bucket. Store your AWS credentials in a text file at `~/.aws/credentials`. This file must use the following format:
  
  ```
    [default]
    aws_access_key_id = YOUR_ACCESS_KEY
    aws_secret_access_key = YOUR_SECRET_KEY
  ```

## Set up the RAG workflow environment

#### Import libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import faiss
import os
import sys

from pathlib import Path

from langchain.text_splitter import RecursiveCharacterTextSplitter

from llama_index.core import VectorStoreIndex, Settings, StorageContext
from llama_index.core.llms import ChatMessage
from llama_index.core.node_parser import LangchainNodeParser
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.openai_like import OpenAILike
from llama_index.readers.s3 import S3Reader
from llama_index.vector_stores.faiss import FaissVectorStore

#### Load config files

In [3]:
# Add root folder of the rag_bootcamp repo to PYTHONPATH
current_dir = Path().resolve()
parent_dir = current_dir.parent
sys.path.insert(0, str(parent_dir))

from utils.load_secrets import load_env_file
load_env_file()

In [4]:
GENERATOR_BASE_URL = os.environ.get("OPENAI_BASE_URL")

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

#### Set up some helper functions

In [5]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.text for i, d in enumerate(docs)]
        )
    )

#### Make sure other necessary items are in place

In [6]:
# Make sure that the AWS credentials are stored in ~/.aws/credentials
try:
    aws_credentials_file = Path.home() / ".aws/credentials"
    assert aws_credentials_file.exists()
except:
    sys.exit(f"Unable to find your AWS credentials file at {aws_credentials_file}. Make sure this file exists and contains your AWS credentials.\n")

# Make sure the AWS credentials are stored in the correct format
try:
    aws_credentials = open(aws_credentials_file, "r").read().strip()
    assert aws_credentials.startswith("[default]")
    assert "aws_access_key_id" in aws_credentials
    assert "aws_secret_access_key" in aws_credentials
except Exception:
    sys.exit(f"""Unable to load AWS credentials. Make sure your ~/.aws/credentials file is in the following format:
[default]
aws_access_key_id = YOUR_ACCESS_KEY
aws_secret_access_key = YOUR_SECRET_KEY\n""")

#### Choose LLM and embedding model

In [7]:
GENERATOR_MODEL_NAME = "Meta-Llama-3.1-8B-Instruct"
EMBEDDING_MODEL_NAME = "BAAI/bge-base-en-v1.5"

## Start with a basic generation request without RAG augmentation

Let's start by asking Llama-3.1 a difficult, domain-specific question we don't expect it to have an answer to. A simple question like "*What is the capital of France?*" is not a good question here, because that's world knowledge that we expect the LLM to know.

Instead, we want to ask it a question that is domain-specific and it won't know the answer to. A good example would be an obscure detail buried deep within a company's annual report. For example:

*How many Vector scholarships in AI were awarded in 2022?*

In [8]:
query = "How many Vector scholarships in AI were awarded in 2022?"

## Now send the query to the open source model using KScope

In [9]:
llm = OpenAILike(
    model=GENERATOR_MODEL_NAME,
    is_chat_model=True,
    temperature=0,
    max_tokens=None,
    api_base=GENERATOR_BASE_URL,
    api_key=OPENAI_API_KEY
)
message = [
    ChatMessage(
        role="user",
        content=query
    )
]
result = llm.chat(message)
print(f"Result: \n\n{result}")

Result: 

assistant: I don't have access to real-time data or specific information about the number of Vector scholarships in AI awarded in 2022. For the most accurate and up-to-date information, I recommend checking directly with Vector Institute or their official website. They would have the most current details on their scholarship programs and awards.


Without additional information, Llama-3.1 is unable to answer the question correctly. **Vector in fact awarded 109 AI scholarships in 2022.** Fortunately, we do have that information available in Vector's 2021-22 Annual Report, which is available in AWS S3 bucket. Let's see how we can use RAG to augment our question with a document search (stored in S3) and get the correct answer.

## Ingestion: Retrieve and store the documents from an Amazon S3 bucket

We've added Vector's 2021-22 Annual Report pdf in an Amazon S3 bucket:

![aws-s3-snapshot](imgs/aws-s3-snapshot.png)

#### Load these documents using S3Reader

Fortunately, there is a simple S3 utility available via [LlamaHub](https://www.llamahub.ai/), a registry of open-source data connectors that you can easily plug into any LlamaIndex application.

In [10]:
loader = S3Reader(bucket='vector-rag-bootcamp-v2')
docs = loader.load_data()
print(f"Number of source documents: {len(docs)}")

Number of source documents: 42


In [11]:
# Split the documents into smaller chunks
parser = LangchainNodeParser(RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=32))
chunks = parser.get_nodes_from_documents(docs)
print(f"Number of text chunks: {len(chunks)}")

Number of text chunks: 228


#### Define the embeddings model

In [12]:
print(f"Setting up the embeddings model...")
embeddings = HuggingFaceEmbedding(
    model_name=EMBEDDING_MODEL_NAME,
    device='cuda',
    trust_remote_code=True,
)

Setting up the embeddings model...


#### Set LLM and embedding model [recommended for LlamaIndex]

In [13]:
Settings.llm = llm
Settings.embed_model = embeddings

## Retrieval: Make the document chunks available via a retriever

In [14]:
def get_embed_model_dim(embed_model):
    embed_out = embed_model.get_text_embedding("Dummy Text")
    return len(embed_out)

faiss_dim = get_embed_model_dim(embeddings)
faiss_index = faiss.IndexFlatL2(faiss_dim)

vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex(chunks, storage_context=storage_context)

In [15]:
retriever = index.as_retriever(similarity_top_k=10)

# Retrieve the most relevant context from the vector store based on the query
retrieved_docs = retriever.retrieve(query)

In [16]:
pretty_print_docs(retrieved_docs)

Document 1:

26 
  VECTOR SCHOLARSHIPS IN 
AI ATTRACT TOP TALENT TO ONTARIO UNIVERSITIES 
109 
Vector Scholarships in AI awarded 
34 
Programs 
13 
Universities 
351 
Scholarships awarded since the 
program launched in 2018 Supported with funding from the Province of Ontario, the Vector Institute Scholarship in Artifcial Intelligence (VSAI) helps Ontario universities to attract the best and brightest students to study in AI-related master’s programs. 
Scholarship recipients connect directly with leading
----------------------------------------------------------------------------------------------------
Document 2:

5 
Annual Report 2021–22 Vector Institute
SPOTLIGHT ON FIVE YEARS OF AI 
LEADERSHIP FOR CANADIANS 
SINCE THE VECTOR INSTITUTE WAS FOUNDED IN 2017: 
2,080+ 
Students have graduated from 
Vector-recognized AI programs and 
study paths $6.2 M 
Scholarship funds committed to 
students in AI programs 3,700+ 
Postings for AI-focused jobs and 
internships ofered on Vector’s 
Digita

## Now send the query to the RAG pipeline

In [17]:
query_engine = RetrieverQueryEngine(retriever=retriever)
result = query_engine.query(query)
print(f"Result: \n\n{result}")

Result: 

109 Vector Scholarships in AI were awarded.


The model provides the correct answer (109) using the retrieved information.